In [1]:
import numpy as np
import cvxopt as cv
import math
from sklearn import svm
TRAIN_FILE = 'fashion_mnist/train.csv'
TEST_FILE = 'fashion_mnist/test.csv'
VAL_FILE = 'fashion_mnist/val.csv'

In [2]:
# Load data from files
data = np.genfromtxt(TRAIN_FILE,delimiter=',')
tdata = np.genfromtxt(TEST_FILE,delimiter=',')
vdata = np.genfromtxt(VAL_FILE,delimiter=',')

In [3]:
C = 1.0
y = 0.05

## Binary Classifier

In [4]:
# Define classes to classify
d = 0
classes = [d, (d+1)%10] # Binary classification 

In [70]:
# change classes to {-1,1}
# change values to [0,1] from [0,255]
newdata = np.asarray([r for r in data if r[-1] in classes])
X = newdata[:,0:784] / 255.0
Y = np.asarray(newdata[:,-1])
minY = np.min(Y)
maxY = np.max(Y)
Y = (Y - (float(minY + maxY)/2.0))/(float(maxY - minY)/2.0)
Y = np.asarray([Y]).astype(float)

In [4]:
# Gaussian Kernel
def gaussian(y,v):
    return math.exp(-y * v)
GAUS = np.vectorize(gaussian)

In [205]:
# linear kernel
Z = Y.T * X
## M x M array Linear kernel
P = np.matmul(Z, Z.T)

In [5]:
# returns alphas as np array
def train(islinear,X,Y):
    M = len(X)
    P = np.eye(M).astype(float)
    if(islinear):
        print("Using Linear Kernel")
        # linear kernel
        Z = Y.T * X
        ## M x M array Linear kernel
        P = np.matmul(Z, Z.T)
    else:
        print("Using Gaussian Kernel")
        for i in range(0,M):
            t = X - X[i]
            P[i] = np.sum(t*t,axis=1)
        P = np.matmul(Y.T,Y) * GAUS(y,P)
    P = cv.matrix(P)
    # column vector with all -1
    Q = cv.matrix(-1 * np.ones((M,1)).astype(float))
    G = cv.matrix(np.vstack((-1 * np.eye(M).astype(float),np.eye(M).astype(float))))
    h = cv.matrix(np.vstack((np.zeros((M,1)).astype(float),C * np.ones((M,1)).astype(float))))
    A = cv.matrix(Y)
    b = cv.matrix([[0]],tc='d')
    print("Finding the optimal solution:")
    solution = cv.solvers.qp(P,Q,G,h,A,b)
    return np.array(solution['x'])

In [6]:
def traingaus(GP,Y,M):
#     M = len(Y)
    print("Training on gaussian kernel")
    P = cv.matrix(np.matmul(Y.T,Y) * GP)
    # column vector with all -1
    Q = cv.matrix(-1 * np.ones((M,1)).astype(float))
    G = cv.matrix(np.vstack((-1 * np.eye(M).astype(float),np.eye(M).astype(float))))
    h = cv.matrix(np.vstack((np.zeros((M,1)).astype(float),C * np.ones((M,1)).astype(float))))
    A = cv.matrix(Y)
    b = cv.matrix([[0]],tc='d')
    print("Finding the optimal solution:")
    solution = cv.solvers.qp(P,Q,G,h,A,b)
    return np.array(solution['x'])

In [112]:
alph = train(True)

Using Linear Kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -3.2916e+02 -9.3878e+03  5e+04  3e+00  3e-12
 1: -2.0018e+02 -5.2391e+03  1e+04  4e-01  2e-12
 2: -9.3627e+01 -1.6503e+03  3e+03  8e-02  1e-12
 3: -5.1870e+01 -7.1316e+02  1e+03  3e-02  8e-13
 4: -2.9393e+01 -4.7263e+02  7e+02  2e-02  5e-13
 5: -1.8616e+01 -2.5482e+02  4e+02  7e-03  4e-13
 6: -1.5942e+01 -8.1551e+01  9e+01  2e-03  4e-13
 7: -1.7889e+01 -4.1759e+01  3e+01  4e-04  4e-13
 8: -1.9475e+01 -3.1857e+01  1e+01  8e-05  4e-13
 9: -1.9419e+01 -2.8941e+01  1e+01  3e-16  4e-13
10: -2.1921e+01 -2.4878e+01  3e+00  9e-16  4e-13
11: -2.2590e+01 -2.3752e+01  1e+00  2e-16  4e-13
12: -2.3023e+01 -2.3203e+01  2e-01  4e-15  4e-13
13: -2.3099e+01 -2.3108e+01  1e-02  2e-15  4e-13
14: -2.3103e+01 -2.3103e+01  2e-04  5e-15  4e-13
15: -2.3103e+01 -2.3103e+01  2e-06  9e-16  4e-13
Optimal solution found.


In [127]:
def getLinearParams(alph):
    fw = alph * Y.T
    w = np.matmul(fw.T,X)
    R = np.matmul(w,X.T)
    maxv = np.min(R)
    minv = np.max(R)
    for i in range(0,len(R[0])):
        if Y[0][i] == 1:
            if(R[0][i] < minv):
                minv = R[0][i]
        else:
            if(R[0][i] > maxv):
                maxv = R[0][i]
    b = -(maxv+minv)/2.0
    nsv = len(np.where(alph > 0.001)[0])
    return (w,b,nsv)

In [132]:
def testlinear(W,b,X):
    return np.matmul(W,X.T) + b

In [129]:
(W,b,nV) = getLinearParams(alph)


In [13]:
tY = tdata[:,-1]
tX = tdata[:,0:784]/255.0

In [14]:
tx = []
ty = []
for i in range(0,len(tX)):
    if tY[i] not in classes:
        continue;
    else:
        ty.append(tY[i])
        tx.append(tX[i])
tX = np.asarray(tx)
tY = np.asarray(ty)

In [226]:

tR = np.matmul(w,tX.T) + b

In [227]:
tot = 0
corr = 0
for i in range(0,len(tR[0])):
    if(tY[i] not in classes):
        continue;
    tot += 1;
    if((tR[0][i]>0 and tY[i]==1) or (tR[0][i]<0 and tY[i]==0)):
        corr +=1;

In [228]:
print(float(corr)/float(tot))

0.982


In [30]:
y = 0.05
P = np.eye(len(X)).astype(float)
for i in range(0,len(X)):
    t = X - X[i]
    P[i] = np.sum(t*t,axis=1)

In [33]:
g = np.vectorize(gaussian)
pp = g(y,P)

In [34]:
# P matrix
pp = np.matmul(Y.T,Y) * pp

In [45]:
A_ = cv.matrix(A)
P_ = cv.matrix(pp)
Q_ = cv.matrix(Q)
G_ = cv.matrix(G)
h_ = cv.matrix(h)
b_ = cv.matrix(B,tc='d')

In [46]:
sol = cv.solvers.qp(P_,Q_,G_,h_,A_,b_)

     pcost       dcost       gap    pres   dres
 0: -1.6492e+02 -6.6070e+03  3e+04  2e+00  1e-15
 1: -1.1407e+02 -3.0686e+03  5e+03  2e-01  2e-15
 2: -1.0459e+02 -7.6952e+02  9e+02  3e-02  2e-15
 3: -1.2684e+02 -3.0203e+02  2e+02  6e-03  2e-15
 4: -1.4133e+02 -2.0101e+02  6e+01  1e-03  1e-15
 5: -1.4945e+02 -1.6809e+02  2e+01  2e-04  1e-15
 6: -1.5144e+02 -1.6280e+02  1e+01  3e-05  1e-15
 7: -1.5364e+02 -1.5728e+02  4e+00  7e-06  1e-15
 8: -1.5443e+02 -1.5550e+02  1e+00  9e-16  1e-15
 9: -1.5478e+02 -1.5492e+02  1e-01  7e-15  1e-15
10: -1.5483e+02 -1.5484e+02  4e-03  1e-14  1e-15
11: -1.5483e+02 -1.5483e+02  6e-05  1e-14  1e-15
Optimal solution found.


In [47]:
alph = np.array(sol['x'])

In [48]:
fW = alph * Y.T

In [50]:
# get b
R = np.ones((len(X),len(X))).astype(float)
for i in range(0,len(X)):
    tt = X - X[i]
    R[i] = np.sum(tt*tt,axis = 1)


In [51]:
R = np.sum(fW* g(y,R), axis=0)

In [54]:
R.shape

(4500,)

In [56]:
minv = 100
maxv = -100
for i in range(0,len(R)):
    if(Y[0][i]==-1 and R[i] > maxv):
        maxv = R[i]
    elif(Y[0][i]==1 and R[i]<minv):
        minv = R[i]
b = -(maxv + minv)/2.0

In [58]:
# Test
tx = np.ones((len(X),len(tX))).astype(float)
for i in range(0,len(X)):
    tt = tX - X[i]
    tx[i] = np.sum(tt*tt,axis = 1)

In [186]:
Gg = g(y,tx)

In [187]:
Gg

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [61]:
R = np.sum(fW * g(y,tx), axis=0) + b

In [62]:
corr = 0
for i in range(0,len(R)):
    if((R[i] >= 0 and tY[i]==1 )or(R[i]<0 and tY[i] ==0)):
        corr +=1

In [63]:
corr/float(len(R))

0.993

In [4]:

clf1 = svm.SVC(kernel='linear')
clf2 = svm.SVC()

In [9]:
clf1.fit(X,Y[0])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [10]:
clf2.fit(X,Y[0])

/home/dhull/.conda/envs/ocr/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [16]:
t2Y = clf2.predict(tX)
t1Y = clf1.predict(tX)

In [17]:
t1Y.shape

(1000,)

In [19]:
lacc = 0.0
gacc = 0.0
for i in range(0,len(t1Y)):
    if((t2Y[i] > 0 and tY[i] > 0) or (t2Y[i]<0 and tY[i]==0)):
        gacc+=1
    if((t1Y[i] > 0 and tY[i] > 0) or (t1Y[i]<0 and tY[i]==0)):
        lacc+=1
lacc /= float(len(t1Y))
gacc /= float(len(t1Y))

In [110]:
clf2.n_support_

array([275, 273], dtype=int32)

## Multi Class Classifier

In [ ]:
# this would be a nC2 array storing alph. (only making it for gaussian)

In [11]:
# get classes in different rows
allX = data[:,0:784] / 255.0
Y = data[:,-1]

In [12]:
X = {}
for i in range(0,len(allX)):
    if Y[i] not in X:
        X[Y[i]] = [allX[i]]
    else:
        X[Y[i]].append(allX[i])

In [13]:
ALPHAxY = np.zeros((10,10,4500,1)).astype(float)
b = np.zeros((10,10)).astype(float)

In [14]:
# Load saved Alphas and b
ALPHAxY = np.load('alphxy.npy')
b = np.load('b.npy')

In [15]:
def getB(ALPHA,GP):
    R = np.sum(ALPHA * GP, axis=0)
    M = len(R)
    ## assuming arangement is such that top layers is Y = -1 and bottom is Y = 1
    maxv = np.max(R[0:M/2])
    minv = np.min(R[M/2:])
    b = -(maxv + minv)/2.0
    return b

In [16]:
def ip(G,X,i):
    G[i] = np.sum((X-X[i])**2, axis=1)
IP = np.vectorize(ip)

In [17]:
import time

In [12]:
# fill upper triangle with alphas
Yi = np.ones(2250).astype(float)
st = time.time()
for i in range(0,10):
    for j in range(i+1,10):
        print("training for "+str(i) + " " + str(j))
        trainX = np.vstack((X[i],X[j]))
        # treating bigger class as 1 and smaller as -1
        trainY = np.asarray([np.hstack((-1.0*Yi,Yi))]).astype(float)
        GP = np.eye(len(trainX)).astype(float)
        M = len(trainX)
        for f in range(0,M):
#             t = trainX - trainX[f]
#             GP[f] = np.sum(t*t,axis=1)
            GP[f] = np.sum((trainX - trainX[f])**2,axis=1)
        GP = GAUS(y,GP)
        ALPHAxY[i][j] = (traingaus(GP,trainY,M) * trainY.T)
        b[i][j] = getB(ALPHAxY[i][j],GP)
        print(time.time()-st)

training for 0 1
Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -1.6492e+02 -6.6070e+03  3e+04  2e+00  4e-15
 1: -1.1407e+02 -3.0686e+03  5e+03  2e-01  4e-15
 2: -1.0459e+02 -7.6952e+02  9e+02  3e-02  4e-15
 3: -1.2684e+02 -3.0203e+02  2e+02  6e-03  4e-15
 4: -1.4133e+02 -2.0101e+02  6e+01  1e-03  3e-15
 5: -1.4945e+02 -1.6809e+02  2e+01  2e-04  3e-15
 6: -1.5144e+02 -1.6280e+02  1e+01  3e-05  3e-15
 7: -1.5364e+02 -1.5728e+02  4e+00  7e-06  3e-15
 8: -1.5443e+02 -1.5550e+02  1e+00  7e-14  3e-15
 9: -1.5478e+02 -1.5492e+02  1e-01  2e-13  3e-15
10: -1.5483e+02 -1.5484e+02  4e-03  5e-14  4e-15
11: -1.5483e+02 -1.5483e+02  6e-05  2e-14  4e-15
Optimal solution found.
131.822613955
training for 0 2
Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -3.7366e+02 -8.4330e+03  4e+04  2e+00  4e-15
 1: -2.8341e+02 -4.5691e+03  7e+03  2e-01  5e-15
 2: -2.6181e+02 -1.1549e+03  1

 4: -1.0877e+02 -1.5571e+02  5e+01  7e-04  2e-15
 5: -1.1554e+02 -1.3286e+02  2e+01  1e-04  2e-15
 6: -1.1887e+02 -1.2384e+02  5e+00  2e-06  2e-15
 7: -1.2010e+02 -1.2121e+02  1e+00  3e-14  2e-15
 8: -1.2044e+02 -1.2063e+02  2e-01  1e-13  2e-15
 9: -1.2051e+02 -1.2052e+02  1e-02  2e-15  2e-15
10: -1.2052e+02 -1.2052e+02  2e-04  9e-14  2e-15
11: -1.2052e+02 -1.2052e+02  9e-06  1e-13  2e-15
Optimal solution found.
1410.99896193
training for 1 5
Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -9.3956e+01 -6.5185e+03  3e+04  2e+00  2e-15
 1: -5.0069e+01 -2.9491e+03  5e+03  2e-01  2e-15
 2: -3.9046e+01 -6.7825e+02  8e+02  3e-02  2e-15
 3: -6.0172e+01 -2.0557e+02  2e+02  5e-03  2e-15
 4: -7.2614e+01 -1.1949e+02  5e+01  1e-03  2e-15
 5: -7.7575e+01 -9.6388e+01  2e+01  1e-04  1e-15
 6: -7.9825e+01 -8.8992e+01  9e+00  2e-13  1e-15
 7: -8.1322e+01 -8.4537e+01  3e+00  2e-13  1e-15
 8: -8.1909e+01 -8.3041e+01  1e+00  7e-14  1e-15
 9: -8

 6: -2.6583e+02 -2.6927e+02  3e+00  2e-05  2e-15
 7: -2.6678e+02 -2.6733e+02  6e-01  3e-06  2e-15
 8: -2.6696e+02 -2.6699e+02  3e-02  1e-07  2e-15
 9: -2.6697e+02 -2.6697e+02  6e-04  2e-09  2e-15
10: -2.6697e+02 -2.6697e+02  1e-05  2e-11  2e-15
Optimal solution found.
2682.92416501
training for 2 9
Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -1.3122e+02 -6.8313e+03  3e+04  2e+00  3e-15
 1: -6.8632e+01 -3.3021e+03  5e+03  2e-01  2e-15
 2: -4.4226e+01 -5.7527e+02  6e+02  2e-02  3e-15
 3: -8.9948e+01 -1.9526e+02  1e+02  2e-03  2e-15
 4: -1.0529e+02 -1.3890e+02  3e+01  6e-04  1e-15
 5: -1.1181e+02 -1.2245e+02  1e+01  1e-04  1e-15
 6: -1.1461e+02 -1.1691e+02  2e+00  8e-06  1e-15
 7: -1.1534e+02 -1.1566e+02  3e-01  3e-07  1e-15
 8: -1.1546e+02 -1.1548e+02  2e-02  4e-09  1e-15
 9: -1.1546e+02 -1.1547e+02  5e-04  1e-10  1e-15
10: -1.1546e+02 -1.1546e+02  1e-05  2e-12  1e-15
Optimal solution found.
2794.06880093
training for 3 4


Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -1.1407e+02 -6.5885e+03  3e+04  2e+00  3e-15
 1: -5.4731e+01 -3.0743e+03  5e+03  2e-01  2e-15
 2: -3.3614e+01 -5.3901e+02  6e+02  2e-02  3e-15
 3: -7.3851e+01 -1.8900e+02  1e+02  3e-03  2e-15
 4: -8.8826e+01 -1.2601e+02  4e+01  7e-04  1e-15
 5: -9.4967e+01 -1.0838e+02  1e+01  2e-04  1e-15
 6: -9.7972e+01 -1.0158e+02  4e+00  2e-05  1e-15
 7: -9.8948e+01 -9.9746e+01  8e-01  2e-06  9e-16
 8: -9.9202e+01 -9.9319e+01  1e-01  2e-07  9e-16
 9: -9.9246e+01 -9.9249e+01  3e-03  3e-09  1e-15
10: -9.9248e+01 -9.9248e+01  7e-05  7e-11  1e-15
Optimal solution found.
4056.41380787
training for 5 6
Training on gaussian kernel
Finding the optimal solution:
     pcost       dcost       gap    pres   dres
 0: -2.0533e+02 -7.5506e+03  3e+04  2e+00  3e-15
 1: -1.2813e+02 -3.9857e+03  6e+03  2e-01  2e-15
 2: -9.5987e+01 -7.8644e+02  8e+02  2e-02  3e-15
 3: -1.4760e+02 -3.3431e+02  2e+02  5e-03  2e-1

In [85]:
# save generated Alphaxy and B
np.save('alphxy.npy',ALPHAxY.astype(np.float32))
np.save('b.npy',b)

In [9]:
# init tx and ty
tX = vdata[:,0:784]/255.0
tY = vdata[:,-1]

In [42]:
# Testing
M = 4500
Votes = np.zeros((45,len(tX))).astype(np.int64)
tx = np.ones((M,len(tX))).astype(float)
st = time.time()
index = 0
for i in range(0,10):
    for j in range(i+1,10):
        print("voting for class "+str(i) + " "+ str(j))
        trainX = np.vstack((X[i],X[j]))
        for k in range(0,M):
            tx[k] = np.sum((tX - trainX[k])**2,axis = 1)
        fW = ALPHAxY[i][j]
        R = np.sum(fW * GAUS(y,tx), axis=0) + b[i][j]
        v = [j if cls>=0 else i for cls in R]
        Votes[index] = v
        index+=1
        print(time.time() - st)
Votes = Votes.T

voting for class 0 1


KeyboardInterrupt: 

In [25]:
vold = Votes


In [27]:
vnew = Votess[0:10]

In [28]:
vnew.shape

(10, 2500)

In [31]:
Votes=np.vstack((vold,vnew)).T

In [90]:
len(Votes[0])

2500

In [32]:
Votes.shape


(2500, 45)

In [24]:
k = 0
for i in range(0,10):
    for j in range(i+1,10):
        k+=1
print(k)

45


In [35]:
Votes = Votes.astype(np.int64)

In [90]:
len(Votes[0])

2500

In [36]:
type(Votes)

numpy.ndarray

In [18]:
# convert to tX * 45
Votes=np.array(Votes).T

In [37]:
# Create final sort from votes
R = np.zeros(len(tX))
for i in range(0,len(tX)):
    tmp = np.bincount(Votes[i])
    R[i] = np.where(tmp == tmp.max())[0][-1]

In [38]:
cnfmat = np.zeros((10,10))
corr = 0.0
for i in range(0,len(tY)):
    if(R[i] ==  tY[i]):
        corr +=1
    cnfmat[int(R[i])][int(tY[i])] +=1

In [90]:
len(Votes[0])

2500

In [39]:
corr/float(len(tY))

0.8464

In [51]:
# Multi class using SK learn 1 vs 1
clf = svm.SVC(kernel='rbf',C=1,gamma=0.05)

In [55]:
clf.fit(allX,Y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [56]:
R = clf.predict(tX)

In [60]:
cnfmat = np.zeros((10,10)).astype(int)
corr = 0.0
for i in range(0,len(R)):
    if(R[i] == tY[i]):
        corr+=1;
    cnfmat[int(R[i])][int(tY[i])]+=1

In [61]:
corr /  float(len(R))

0.8808

SVM ACC -> 0.8808

In [45]:
Cs = [1e-5,1e-3,1,5,10]

In [ ]:
# K-cross validation
# assuming data is already shuffled
k = 5
BS = allX.shape[0]/k
# kX = np.zeros((k,allX.shape[0]/k,allX.shape[1]))
# for i in range(0,k):
#     kX[i] = allX[i*BS:(i+1)*BS]
acc = []
st = time.time()
for c in Cs:
    correct = 0.0
    clf = svm.SVC(kernel='rbf',C=c,gamma=0.05)
    # Treat i-th block as validation set
    for i in range(0,k):
        trainX = np.vstack((allX[:i*BS],allX[(i+1)*BS:]))
        trainY = np.hstack((Y[:i*BS],Y[(i+1)*BS:]))
        print(trainX.shape)
        print(trainY.shape)
        clf.fit(trainX,trainY)
        print("Predicting now:")
        R = clf.predict(allX[i*BS:(i+1)*BS])
        for j in range(0,len(R)):
            if(R[j]==Y[i*BS + j]):
                correct+=1
        print("Accuracy for this set: "+str(correct/float((i+1)*BS)))
        print(time.time() - st)
    accuracy = float(correct)/float(len(allX))
    acc.append(accuracy)
    print(accuracy)

(18000, 784)
(18000,)


In [ ]:
tacc
for c in Cs:
    print("for c = "+str(c)+":")
    correct = 0.0
    clf = svm.SVC(kernel='rbf',C=c,gamma=0.05)
    clf.fit(allX,Y)
    R=clf.predict(tX)
    for i in range(0,len(tX)):
        if(R[i]==tY[i]):
            correct+=1
    acc = float(correct)/float(len(tY))
    tacc.append(acc)
    print(acc)